# 处理数据
PyTorch 提供了两种用于处理数据的基本组件：torch.utils.data.DataLoader 和 torch.utils.data.Dataset。 
Dataset 存储样本及其对应的标签，而 DataLoader 在 Dataset 周围包装一个可迭代对象。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供了特定领域的库，例如 TorchText、TorchVision 和 TorchAudio，它们都包含数据集。在本教程中，我们将使用 TorchVision 数据集。

torchvision.datasets 模块包含许多现实世界视觉数据的 Dataset 对象，例如 CIFAR、COCO (完整列表在此)。在本教程中，我们使用 FashionMNIST 数据集。每个 TorchVision Dataset 包含两个参数：transform 和 target_transform，分别用于修改样本和标签。

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open dataset
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将 Dataset 作为参数传递给 DataLoader。这将包装一个可迭代对象，用于遍历我们的数据集，并支持自动批处理、采样、洗牌和多进程数据加载。在这里，我们定义了一个批次大小为 64，即数据加载器可迭代对象中的每个元素将返回一个包含 64 个特征和标签的批次。

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shaper of X [N, C, H, W]: {X.shape}")
    print(f"Shaper of y: {y.shape}")
    break


Shaper of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shaper of y: torch.Size([64])


# 创建模型
为了在 PyTorch 中定义一个神经网络，我们创建一个继承自 nn.Module 的类。我们在 __init__ 函数中定义网络的层，并在 forward 函数中指定数据如何通过网络传递。为了加速神经网络中的操作，我们将它移动到 GPU 或 MPS（如果可用）。

In [4]:
# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 优化模型参数
为了训练模型，我们需要一个损失函数和一个优化器。

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型会对训练数据集（分批次输入）进行预测，并将预测误差反向传播以调整模型参数。

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还会检查模型在测试数据集上的表现，以确保它正在学习。

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程会进行多次迭代（epochs）。在每个 epoch 中，模型都会学习参数以做出更好的预测。我们会在每个 epoch 打印模型的准确率和损失；我们希望看到准确率随着每个 epoch 的增加而增加，损失随着每个 epoch 的增加而减少。

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309987  [   64/60000]


loss: 2.293439  [ 6464/60000]
loss: 2.270618  [12864/60000]
loss: 2.263408  [19264/60000]
loss: 2.240910  [25664/60000]
loss: 2.221165  [32064/60000]
loss: 2.231277  [38464/60000]
loss: 2.199440  [44864/60000]
loss: 2.198711  [51264/60000]
loss: 2.168355  [57664/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 2.156498 

Epoch 2
-------------------------------
loss: 2.176653  [   64/60000]
loss: 2.159504  [ 6464/60000]
loss: 2.102727  [12864/60000]
loss: 2.120164  [19264/60000]
loss: 2.060859  [25664/60000]
loss: 2.004323  [32064/60000]
loss: 2.036845  [38464/60000]
loss: 1.958220  [44864/60000]
loss: 1.973823  [51264/60000]
loss: 1.901141  [57664/60000]
Test Error: 
 Accuracy: 50.3%, Avg loss: 1.893217 

Epoch 3
-------------------------------
loss: 1.932343  [   64/60000]
loss: 1.899277  [ 6464/60000]
loss: 1.785169  [12864/60000]
loss: 1.832254  [19264/60000]
loss: 1.708182  [25664/60000]
loss: 1.662472  [32064/60000]
loss: 1.685528  [38464/60000]
loss: 1.591028  [44864/60000]
loss: 

# 保存模型
保存模型的一种常见方法是序列化内部状态字典（包含模型参数）。

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

现在可以使用此模型进行预测。

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
